In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
read_csv=pd.read_csv('C:/Users/dell/Desktop/DATA7901/Final_proposal/dataset_space_API_past_launches_raw.csv')
read_csv.dtypes

FlightNumber        int64
Date               object
BoosterVersion     object
PayloadMass       float64
Orbit              object
LaunchSite         object
Outcome            object
Flights             int64
GridFins             bool
Reused               bool
Legs                 bool
LandingPad         object
Block             float64
ReusedCount         int64
Serial             object
Longitude         float64
Latitude          float64
dtype: object

In [2]:
#Data wrangling
falcon9_data = read_csv[read_csv['BoosterVersion'] == 'Falcon 9']
mean_value=round(falcon9_data['PayloadMass'].mean(),2)
falcon9_data_new= falcon9_data.loc[:,'PayloadMass'].fillna(mean_value,inplace=True)
falcon9_data

D:\Program\Anaconda\lib\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
4,6,2010-06-04,Falcon 9,6123.55,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857
5,8,2012-05-22,Falcon 9,525.00,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857
6,10,2013-03-01,Falcon 9,677.00,ISS,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857
7,11,2013-09-29,Falcon 9,500.00,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093
8,12,2013-12-03,Falcon 9,3170.00,GTO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,102,2020-09-03,Falcon 9,15600.00,VLEO,KSC LC 39A,True ASDS,2,True,True,True,5e9e3032383ecb6bb234e7ca,5.0,12,B1060,-80.603956,28.608058
90,103,2020-10-06,Falcon 9,15600.00,VLEO,KSC LC 39A,True ASDS,3,True,True,True,5e9e3032383ecb6bb234e7ca,5.0,12,B1058,-80.603956,28.608058
91,104,2020-10-18,Falcon 9,15600.00,VLEO,KSC LC 39A,True ASDS,6,True,True,True,5e9e3032383ecb6bb234e7ca,5.0,12,B1051,-80.603956,28.608058
92,105,2020-10-24,Falcon 9,15600.00,VLEO,CCSFS SLC 40,True ASDS,3,True,True,True,5e9e3033383ecbb9e534e7cc,5.0,12,B1060,-80.577366,28.561857


In [3]:
# the number and occurrence of each orbit
df1=falcon9_data.groupby(['BoosterVersion','Orbit'])['Orbit'].count().sort_values(ascending=False)
df1

BoosterVersion  Orbit
Falcon 9        GTO      27
                ISS      21
                VLEO     14
                PO        9
                LEO       7
                SSO       5
                MEO       3
                ES-L1     1
                GEO       1
                HEO       1
                SO        1
Name: Orbit, dtype: int64

In [4]:
#the number and occurrence of mission outcome per orbit type
df2=falcon9_data.groupby(['Orbit','Outcome'])['Outcome'].count().reset_index(name='Count').sort_values(['Orbit','Count'], ascending=False)
print(df2)

    Orbit      Outcome  Count
30   VLEO    True ASDS     12
29   VLEO   False ASDS      2
28    SSO    True RTLS      3
27    SSO    True ASDS      2
26     SO    None None      1
24     PO    True ASDS      5
21     PO   False ASDS      1
22     PO  False Ocean      1
23     PO    None None      1
25     PO   True Ocean      1
20    MEO    True ASDS      2
19    MEO    None None      1
18    LEO    True RTLS      4
16    LEO    None None      2
17    LEO   True Ocean      1
15    ISS    True RTLS      7
13    ISS    True ASDS      5
12    ISS    None None      3
8     ISS   False ASDS      2
9     ISS  False Ocean      1
10    ISS   False RTLS      1
11    ISS    None ASDS      1
14    ISS   True Ocean      1
7     HEO    True ASDS      1
5     GTO    True ASDS     13
4     GTO    None None     11
2     GTO   False ASDS      1
3     GTO    None ASDS      1
6     GTO   True Ocean      1
1     GEO    True ASDS      1
0   ES-L1   True Ocean      1


In [5]:
#Classfiy successful landing and fail landing
falcon9_data['Outcome'].astype(str)
landing_status=[]
for row in falcon9_data['Outcome']:
        if row in ("True Ocean", "True RTLS","True ASDS"):landing_status.append("Success")
        elif row in ('False Ocean','False RTLS','False ASDS','None ASDS','None None'):landing_status.append("Fail")
        else: landing_status.append("No Rating")
falcon9_data['LandingStatus'] = landing_status
falcon9_data['LandingStatus'].astype(str)
falcon9_data

C:\Users\dell\AppData\Local\Temp/ipykernel_20036/712157466.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  falcon9_data['LandingStatus'] = landing_status


,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,LandingStatus
4,6,2010-06-04,Falcon 9,6123.55,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857,Fail
5,8,2012-05-22,Falcon 9,525.00,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857,Fail
6,10,2013-03-01,Falcon 9,677.00,ISS,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857,Fail
7,11,2013-09-29,Falcon 9,500.00,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093,Fail
8,12,2013-12-03,Falcon 9,3170.00,GTO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857,Fail
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,102,2020-09-03,Falcon 9,15600.00,VLEO,KSC LC 39A,True ASDS,2,True,True,True,5e9e3032383ecb6bb234e7ca,5.0,12,B1060,-80.603956,28.608058,Success
90,103,2020-10-06,Falcon 9,15600.00,VLEO,KSC LC 39A,True ASDS,3,True,True,True,5e9e3032383ecb6bb234e7ca,5.0,12,B1058,-80.603956,28.608058,Success
91,104,2020-10-18,Falcon 9,15600.00,VLEO,KSC LC 39A,True ASDS,6,True,True,True,5e9e3032383ecb6bb234e7ca,5.0,12,B1051,-80.603956,28.608058,Success
92,105,2020-10-24,Falcon 9,15600.00,VLEO,CCSFS SLC 40,True ASDS,3,True,True,True,5e9e3033383ecbb9e534e7cc,5.0,12,B1060,-80.577366,28.561857,Success


In [6]:
#Success and fail launch
df3=falcon9_data.groupby(['BoosterVersion','LandingStatus'])['LandingStatus'].count()
print(df3)
#The number of Each orbit's successful and fail landing 
df4=falcon9_data.groupby(['Orbit','LandingStatus'])['LandingStatus'].count().reset_index(name='Count').sort_values(['Orbit','Count'], ascending=False)
print(df4)

BoosterVersion  LandingStatus
Falcon 9        Fail             30
                Success          60
Name: LandingStatus, dtype: int64
    Orbit LandingStatus  Count
16   VLEO       Success     12
15   VLEO          Fail      2
14    SSO       Success      5
13     SO          Fail      1
12     PO       Success      6
11     PO          Fail      3
10    MEO       Success      2
9     MEO          Fail      1
8     LEO       Success      5
7     LEO          Fail      2
6     ISS       Success     13
5     ISS          Fail      8
4     HEO       Success      1
3     GTO       Success     14
2     GTO          Fail     13
1     GEO       Success      1
0   ES-L1       Success      1
